# Working with Agent Jobs with dbatools

You can use dbatools to administer your Agent jobs. Make sure that you have first run your create container notebook

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

The cell block below will create the variables that you will need for the rest of the notebook

You can run the code by using the arrow keys and shift and enter

In [ ]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}
if( $Host -and $Host.UI -and $Host.UI.RawUI ) {
    $rawUI = $Host.UI.RawUI
    $oldSize = $rawUI.BufferSize
    $typeName = $oldSize.GetType( ).FullName
    $newSize = New-Object $typeName (500, $oldSize.Height)
    $rawUI.BufferSize = $newSize
  }

## List the Agent jobs on the instance

You can very quickly list all the agent jobs on an instance with dbatools

In [ ]:
Get-DbaAgentJob -SqlInstance $sql1 | Format-Table

You can get info about a particular job

In [ ]:
Get-DbaAgentJob -SqlInstance $sql1 -Job 'The Beard is Important'

Job Steps

In [ ]:
Get-DbaAgentJobStep -SqlInstance $SQL1 -Job 'The Beard is Important'

You can remove jobs 

**PLEASE DON'T DO THIS IN PRODUCTION**

In [ ]:
Get-DbaAgentJob -SqlInstance $SQL2 | Remove-DbaAgentJob -Confirm:$false

We have no jobs on SQL2

In [ ]:
 $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
Get-DbaAgentJob -SqlInstance $SQL2

If you have run through the notebooks, you might still have the Availability Group that was created in Notebook 03

You would want your Agent Jobs to be the same on all replicas so that if you failover everything works

dbatools has you covered

In [ ]:
Copy-DbaAgentJob -Source $SQL1 -Destination $SQL2

and just like that all jobs are on SQL2

In [ ]:
 $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
Get-DbaAgentJob -SqlInstance $SQL2|Format-Table -AutoSize

# Clean Up

The 99-CleanUpContainers notebook will remove the containers, files and directory - it will leave the image so you do not have to download it again!